<a href="https://colab.research.google.com/github/Raida03/AI360s2fall2024/blob/main/Lab07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install grad-cam


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.4-py3-none-any.whl size=39648 sha256=a41230dc7dfb22691b34cde5c8f5185475d41f92adcb73e8e3972c554577dced
  Stored in directory: /root/.cache/pip/wheels/50/b0/82/1f97b5348c7fe9f0ce0ba18497202cafa5dec4562bd5292680
Successfully built grad-cam


In [5]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import numpy as np
from torchvision import datasets, transforms


In [6]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import zipfile
import os

dataset_path = "/content/drive/MyDrive/caltech-101.zip"
extract_path = "/content/drive/MyDrive/caltech-101"  # Directory to extract to

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [11]:
import shutil
shutil.rmtree(os.path.join(extract_path, '__MACOSX'), ignore_errors=True)



In [13]:
import os
for root, dirs, files in os.walk(extract_path):
    print(f"Root: {root}, Files: {files}")



Root: /content/drive/MyDrive/caltech-101, Files: []
Root: /content/drive/MyDrive/caltech-101/caltech-101, Files: ['101_ObjectCategories.tar.gz', 'show_annotation.m', 'Annotations.tar']


In [ ]:
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])


In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)

In [ ]:
vgg19 = models.vgg19(pretrained=True)
vgg19.classifier[6] = nn.Linear(4096, 101)

resnet50 = models.resnet50(pretrained=True)
resnet50.fc = nn.Linear(2048, 101)

efficientnet_b0 = models.efficientnet_b0(pretrained=True)
efficientnet_b0.classifier[1] = nn.Linear(1280, 101)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_vgg = optim.Adam(vgg19.parameters(), lr=0.001)
optimizer_resnet = optim.Adam(resnet50.parameters(), lr=0.001)
optimizer_efficientnet = optim.Adam(efficientnet_b0.parameters(), lr=0.001)

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.numpy())
            y_true.extend(labels.numpy())

    cm = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(report)

In [ ]:
print("Evaluating VGG19...")
evaluate_model(vgg19, test_loader)

print("Evaluating ResNet50...")
evaluate_model(resnet50, test_loader)

print("Evaluating EfficientNet-B0...")
evaluate_model(efficientnet_b0, test_loader)

print("Evaluation completed!")

In [ ]:
def apply_grad_cam(model, target_layer, test_loader):
    cam = GradCAM(model=model, target_layers=[target_layer])

    for images, labels in test_loader:
        images_np = images[0].permute(1, 2, 0).numpy()
        grayscale_cam = cam(input_tensor=images.unsqueeze(0), target_category=labels[0].item())
        cam_image = show_cam_on_image(images_np, grayscale_cam[0], use_rgb=True)

        plt.imshow(cam_image)
        plt.axis('off')
        plt.show()
        break

In [ ]:
print("Applying Grad-CAM on VGG19...")
apply_grad_cam(vgg19, vgg19.features[-1], test_loader)

# Apply Grad-CAM on ResNet50
print("Applying Grad-CAM on ResNet50...")
apply_grad_cam(resnet50, resnet50.layer4[-1], test_loader)

# Apply Grad-CAM on EfficientNet-B0
print("Applying Grad-CAM on EfficientNet-B0...")
apply_grad_cam(efficientnet_b0, efficientnet_b0.features[-1], test_loader)

print("Grad-CAM visualization completed!")
